In [1]:
%load_ext cudf.pandas

# Binary data without lazy loading


**Author**: Aureliana Barghini ([B-Open](https://www.bopen.eu/))

## BackendEntrypoint
Implement a subclass of `BackendEntrypoint` that expose a method `open_dataset`:

```python
from xarray.backends import BackendEntrypoint

class MyBackendEntrypoint(BackendEntrypoint):
    def open_dataset(
        self,
        filename_or_obj,
        *,
        drop_variables=None,
    ):

        return my_open_dataset(filename_or_obj, drop_variables=drop_variables)

```

## BackendEntrypoint integration
Declare this class as an external plugin in your `setup.py`:

```python
setuptools.setup(
    ...
    entry_points={
        'xarray.backends': ['engine_name=package.module:my_backendentrypoint'],
    },
)
```
or pass it in `xr.open_dataset`:

```python
xr.open_dataset(..., engine=MyBackendEntrypoint)
```

## Example backend for binary files

In [2]:
%%cudf.pandas.profile

import numpy as np
import xarray as xr

                                                                                              
                                  Total time elapsed: 1.602 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

### Create sample files

In [3]:
%%cudf.pandas.profile

arr = np.arange(30000000, dtype=np.int64)
with open("foo.bin", "w") as f:
    arr.tofile(f)

arr = np.arange(30000000, dtype=np.float64)
with open("foo_float.bin", "w") as f:
    arr.tofile(f)

                                                                                              
                                  Total time elapsed: 0.887 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

### Define the entrypoint
Example of backend to open binary files

In [4]:
%%cudf.pandas.profile

class BinaryBackend(xr.backends.BackendEntrypoint):
    def open_dataset(
        self,
        filename_or_obj,
        *,
        drop_variables=None,
        # backend specific parameter
        dtype=np.int64,
    ):
        with open(filename_or_obj) as f:
            arr = np.fromfile(f, dtype)

        var = xr.Variable(dims=("x"), data=arr)
        coords = {"x": np.arange(arr.size) * 10}
        return xr.Dataset({"foo": var}, coords=coords)

                                                                                              
                                  Total time elapsed: 0.034 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

### It Works! 
But it may be memory demanding

In [5]:
%%cudf.pandas.profile

arr = xr.open_dataarray("foo.bin", engine=BinaryBackend)
arr

<xarray.DataArray 'foo' (x: 30000000)>
[30000000 values with dtype=int64]
Coordinates:
  * x        (x) int64 0 10 20 30 40 ... 299999960 299999970 299999980 299999990

                                                                                                            
                                         Total time elapsed: 3.218 seconds                                  
                                       96 GPU function calls in 1.844 seconds                               
                                       0 CPU function calls in 0.000 seconds                                
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Index                  │ 1          │ 1.803       │ 1.803       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__     │ 85         │ 0.026       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy        │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__ │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__getitem__ │ 6          │ 0.005       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__repr__    │ 1          │ 0.009       │ 0.009       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [6]:
%%cudf.pandas.profile

arr = xr.open_dataarray("foo_float.bin", engine=BinaryBackend, dtype=np.float64)
arr

<xarray.DataArray 'foo' (x: 30000000)>
[30000000 values with dtype=float64]
Coordinates:
  * x        (x) int64 0 10 20 30 40 ... 299999960 299999970 299999980 299999990

                                                                                                            
                                         Total time elapsed: 0.595 seconds                                  
                                       96 GPU function calls in 0.064 seconds                               
                                       0 CPU function calls in 0.000 seconds                                
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Index                  │ 1          │ 0.025       │ 0.025       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__     │ 85         │ 0.025       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy        │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__ │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__getitem__ │ 6          │ 0.005       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__repr__    │ 1          │ 0.008       │ 0.008       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [7]:
%%cudf.pandas.profile

arr.sel(x=slice(0, 100))

<xarray.DataArray 'foo' (x: 11)>
[11 values with dtype=float64]
Coordinates:
  * x        (x) int64 0 10 20 30 40 50 60 70 80 90 100

                                                                                                              
                                          Total time elapsed: 0.439 seconds                                   
                                        89 GPU function calls in 0.036 seconds                                
                                        1 CPU function calls in 0.175 seconds                                 
                                                                                                              
                                                        Stats                                                 
                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                 ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Int64Index.__len__       │ 80         │ 0.023       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.slice_indexer │ 0          │ 0.000       │ 0.000       │ 1          │ 0.175       │ 0.175       │
│ Int64Index.__getitem__   │ 7          │ 0.009       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__repr__      │ 1          │ 0.004       │ 0.004       │ 0          │ 0.000       │ 0.000       │
└──────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- Int64Index.slice_indexer

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=345416;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.